# Pipeline for calculating overbite

## Initialization
We assume that the images used for our pipeline is transformed and aligned to the standard of the images used in training. This can be done with a preprocessing function.

In [5]:
# Importing packages
import os
import torch
import re
import torchvision.transforms as T
from collections import defaultdict
from PIL import Image

In [ ]:
def group_images_by_prefix(folder_path):
    """
    Scans folder_path for image files (.jpg/.jpeg/.png), and groups them by the prefix
    (stem without the last '_' segment). Returns only groups of exactly four images.
    """
    # List all image files
    files = [
        f for f in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, f))
        and f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]
    # group by prefix
    groups = defaultdict(list)
    for fn in files:
        # Fjern filendelsen og få filnavnets “stamme”
        stem, _ = os.path.splitext(fn)
        # Del kun på det første '_' og tag den første del
        prefix = stem.split('_', 1)[0]
        # Fjern alt, der ikke er A–Z, a–z eller 0–9
        key = re.sub(r'[^A-Za-z0-9]', '', prefix)
        # Tilføj til gruppen
        groups[key].append(os.path.join(folder_path, fn))
    # 3) keep only groups of exactly 4
    return {
        key: sorted(paths)
        for key, paths in groups.items()
        if len(paths) == 4
    }

def load_model(model_path, device='cpu'):
    """Load the TorchScript Keypoint R-CNN once."""
    model = torch.jit.load(model_path, map_location=device)
    model.eval()
    return model

def infer_on_quadruple(image_paths, model, device='cpu'):
    """
    Runs inference on exactly four images at once,
    returns a dict { filename: (rows_list, cols_list) }.
    """
    inputs = []
    for path in image_paths:
        img = Image.open(path).convert("RGB")
        tensor = transform(img).to(device)
        inputs.append({
            "image": tensor,
            "height": tensor.shape[1],
            "width":  tensor.shape[2],
        })

    with torch.no_grad():
        outputs = model(inputs)

    results = {}
    for path, out in zip(image_paths, outputs):
        kpts = out["instances"].pred_keypoints  # [N_instances, K, 3]
        coords = kpts[..., :2].round().to(torch.int64).view(-1, 2)
        xs = coords[:, 0].tolist()
        ys = coords[:, 1].tolist()
        results[os.path.basename(path)] = (ys, xs)

    return results

def estimation_of_keypoints(folder_path, model_path, device='cpu'):
    """
    1) Groups images in folder_path into quadruples by filename prefix.
    2) Runs keypoint inference on each quadruple.
    3) Returns a dict:
         {
           "group_prefix1": {
               "img1.jpg": ([y1,y2,…], [x1,x2,…]),
               "img2.jpg": …,
               …
           },
           "group_prefix2": { … },
           …
         }
    """
    groups = group_images_by_prefix(folder_path)
    if not groups:
        print("No groups of exactly four images found.")
        return {}

    model     = load_model(model_path, device)

    all_results = {}
    for prefix, paths in groups.items():
        all_results[prefix] = infer_on_quadruple(paths, model, device)

    return all_results



FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/image_folder'

In [ ]:
# Example usage:
folder      = "/path/to/your/image_folder"
model_file  = "detectron2_traced.pt"
device      = "cuda"  # or "cpu"
results = estimation_of_keypoints(folder, model_file, device)
for group, inf in results.items():
    print(f"\nGroup: {group}")
    for img_name, (rows, cols) in inf.items():
        print(f"  {img_name}: {len(rows)} keypoints — sample: {list(zip(rows,cols))[:5]}")